<a href="https://colab.research.google.com/github/qianzhou1982/Demo/blob/master/Gestionnaires_de_contexte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gestionnaires de contexte

## Implémentation d'un gestionnaire de contexte avec les méthodes magiques

Créez une classe `StdoutRedirector` qui sera utilisable comme un gestionnaire de contexte. Vous pouvez consulter la documentation du type [`ContextManager`](https://docs.python.org/fr/3/library/stdtypes.html#typecontextmanager). Cette classe stockera la valeur de `sys.stdout` à l'entrée dans le contexte, et la remplacera par l'argument qui lui est donné à sa création. À la sortie du contexte, l'ancienne sortie standard devra être restaurée.

In [ ]:
# Votre code ici

### Solution

In [ ]:
import io
import sys


class StdoutRedirector:
  def __init__(self, new_stdout):
    self.new_stdout = new_stdout

  def __enter__(self):
    self.old_stdout = sys.stdout
    sys.stdout = self.new_stdout

  def __exit__(self, exc_type, exc_val, exc_tb):
    sys.stdout = self.old_stdout
    return False


with io.StringIO() as string_io:
  with StdoutRedirector(string_io):
    print("Hello World")
  print(f"Valeur de la StringIO : {string_io.getvalue()}")

## Implémentation d'un gestionnaire de contexte avec un générateur

Recréez le contexte StdoutRedirector mais cette fois à l'aide de la méthode [`contextlib.contextmanager`](https://docs.python.org/fr/3/library/contextlib.html#contextlib.contextmanager), en utilisant un générateur.

In [ ]:
# Votre code ici

### Solution

In [ ]:
import contextlib
import io
import sys


@contextlib.contextmanager
def StdoutRedirector(new_stdout):
  try:
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    yield
  finally:
    sys.stdout = old_stdout


with io.StringIO() as string_io:
  with StdoutRedirector(string_io):
    print("Hello World")
  print(f"Valeur de la StringIO : {string_io.getvalue()}")

## Implémentation d'un contexte qui gère la fermeture d'une ressource

Implémentez avec et sans `contextlib.contextmanager` un gestionnaire de contexte qui ferme une ressource qui dispose d'une méthode `close` qu'on lui donne en argument quoi qu'il arrive à la fin du contexte.

In [ ]:
# Votre code ici

### Solution

In [ ]:
import contextlib


@contextlib.contextmanager
def closing(resource):
  try:
    yield
  finally:
    resource.close()


class Closing:
  def __init__(self, resource):
    self.resource = resource

  def __enter__(self):
    pass

  def __exit__(self, exc_type, exc_val, exc_tb):
    self.resource.close()
    return False


class Resource:
  def close(self):
    print("Fermée")

resource = Resource()
with Closing(resource):
  pass

## Réimplémentation du décorateur `contextlib.contextmanager` (difficile)

À l'aide d'un décorateur, qui créera une classe avec les méthodes `__init__`, `__enter__` et `__exit__`, réimplémentez le décorateur [`contextlib.contextmanager`](https://docs.python.org/fr/3/library/contextlib.html#contextlib.contextmanager).

In [ ]:
# Votre code ici

### Solution

In [ ]:
import io

def contextmanager(generator):
  class Wrapper:
    def __init__(self, *args, **kwargs):
      self.generator = generator(*args, **kwargs)

    def __enter__(self):
      return next(self.generator)

    def __exit__(self, exc_type, exc_val, exc_tb):
      try:
        next(self.generator)
      except StopIteration:
        pass
      else:
        raise RuntimeError("Generator used as a context should yield only once")
      False

  return Wrapper

@contextmanager
def StdoutRedirector(new_stdout):
  try:
    old_stdout = sys.stdout
    sys.stdout = new_stdout
    yield
  finally:
    sys.stdout = old_stdout

string_io = io.StringIO()

with StdoutRedirector(string_io):
  print("Hello World!")

print("Hello Nantes!")

string_io.getvalue()